In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

import time
import os
import numpy as np

import tensorflow as tf

modelfileName = "CS286-Word2Vec.model"

dirNameInput = os.getcwd() + '/Word2VecData_Input'
dirNameOutput = os.getcwd() + '/Word2VecData_Output'

Word2Vec_model = None

X = None
Y = None
x_train = None
y_train = None
x_test = None
y_test = None

model = None


In [ ]:
def createDataFiles():
    import time
    start = time.time()
    stride = 2
    messageSize = 5
    
    # For Simulated data
    fileInput = open(dirNameInput + "/Word2VecDataSimulated.txt", "w")
    fileOutput = open(dirNameOutput + "/Word2VecDataSimulated.txt", "w")
    fileNamesSimulated = ['Speed20_40.txt', 'Speed40_60.txt']
    arrSimulated = [1, 2]

    for i in range(len(fileNamesSimulated)):
        with open(fileNamesSimulated[i]) as f:
            content = f.readlines()

        # Remove the extra spaces from the sentences
        content = [x.strip() for x in content]

        cntCntr = 0
        while cntCntr < len(content):
            iteration = 0
            currMessagePackets = ""

            while iteration < messageSize and cntCntr < len(content):
                # Remove the noise from the files
                DATA = content[cntCntr][15:38]

                # this is the whole message
                parts = DATA.split()

                tempo = ""
                for j in range(8):
                    if j >= len(parts) or parts[j] == "  ":
                        tempo += "00"
                    else:
                        tempo += parts[j]

                currMessagePackets += str(tempo) + " "
                cntCntr += 1
                iteration += 1

            if cntCntr < len(content):
                cntCntr -= stride
            else:
                break
            fileInput.write(currMessagePackets + "\n")
            fileOutput.write(str(arrSimulated[i]) + "\n")

    fileInput.close()
    fileOutput.close()
    # To only work with actual data end commenting till here
    end = time.time()

    print("Awesome !!! File processing done !!!")
    print("Total Time for file processing ---> ", end - start)


In [ ]:
class IteratingClass:
    def __init__(self, dirName):
        self.dirName = dirName

    def __iter__(self):
        for fName in os.listdir(self.dirName):
            for line in open(os.path.join(self.dirName, fName)):
                yield line.split()

def createWord2VecModels():
    start = time.time()
    dataDirec = IteratingClass(dirNameInput)
    model = Word2Vec(dataDirec, size=200, window=5, min_count=1, workers=8)
    model.save(modelfileName)
    end = time.time()
    print("\n\nTraining Successful for Word2Vec Model!!!")
    print("Total Time for Word2Vec model -> ", (end - start))


In [ ]:
def train():
    createDataFiles()
    createWord2VecModels()

In [ ]:
train()

In [ ]:

# Convert the data into Numpy Arrays
def getNumpyArray():
    start = time.time()
    global X
    global Y
    global dirNameInput
    Word2Vec_model = Word2Vec.load(modelfileName)
    X = None
    Y = None
    print("Processing started ")
    result = []
    for fName in os.listdir(dirNameInput):
        for line in open(os.path.join(dirNameInput, fName)):
            parts = line.split()
            currV = Word2Vec_model.wv[parts[0]]
            currV.setflags(write=1)
            for i in range(1, len(parts)):
                currV += Word2Vec_model.wv[parts[i]]
            currV /= len(parts)
            result.append(np.array(currV))
    X = np.array(result)
               
    print("Processing started for output ")
    result = []
    for fName in os.listdir(dirNameOutput):
        for line in open(os.path.join(dirNameOutput, fName)):
            currV = int(line)
            result.append(np.array(currV))

    Y = np.array(result)        
    
    print(X.shape)    
    print(Y.shape)    
    end = time.time()
    print("total time -> ", end -start)





In [ ]:
def createDNNModel():
    global model     
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(3, activation=tf.nn.softmax))

    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])
    

    

    

In [ ]:
getNumpyArray()
# Commnet this line for your part
createDNNModel()
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size=0.30, random_state=1)


In [ ]:
def executeModel(eph):
    start = time.time()
    model.compile(optimizer='adam',
                 loss='sparse_categorical_crossentropy',
                 metrics=['accuracy'])

    model.fit(x_train, y_train, epochs=eph)
    end = time.time()
    print("Total Time -> ", (end - start))
    
    val_loss, val_acc = model.evaluate(x_test, y_test)
    print(val_loss)
    print(val_acc)

In [ ]:
start = time.time()
executeModel(50)
end = time.time()
print("Total Time -->", end - start)

In [ ]:
def LSTMModel(eph):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM((1), batch_input_shape=(None, 200, 1), return_sequences=True))
    #model.add(tf.keras.layers.LSTM(5 ,input_shape=(1,200)))
    #model.add(tf.keras.layers.Dense(1, activation='sigmoid'))


    print ('Compiling...')
    #model.compile(loss='binary_crossentropy',optimizer='rmsprop',metrics=['accuracy'])
    model.compile(loss='mean_absolute_error',optimizer='adam',metrics=['accuracy'])
    #y_train.reshape(200,1)
    model.fit(x_train, y_train, batch_size = 100 , epochs=eph, verbose = 1)
    
    end = time.time()
    print("Total Time -> ", (end - start))
    


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y,test_size=0.30, random_state=1)
print(x_train.shape)
print(y_train.shape)
x_train = x_train.reshape(837,200,1)
y_train = y_train.reshape(837,1)
LSTMModel(10)

In [ ]:
score, acc = model.evaluate(x_test, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

In [ ]:
def evaluateWordToVec():
    Word2Vec_model = Word2Vec.load(modelfileName)
    print("Loaded successfully")
    #print(model.wv['FEB0FF999E500000'])
    print(len(Word2Vec_model.wv['FEB0FF999E500000']))

    #print(model.wv['000200000000002A'])
    print(len(Word2Vec_model.wv['000200000000002A']))
    
    #print(model.wv['FEB0FF999E500000'] + model.wv['000200000000002A'])
    
    v = Word2Vec_model.wv['FEB0FF999E500000'] + Word2Vec_model.wv['000200000000002A']
    print(type(v))
    #print(v/2)
evaluateWordToVec()